## Create Default Folio

In [ ]:
# Bootstrapping
import pandas as pd

from app.bootstrap import reload_config
from mock.folio_setup import ensure_folio_exists

config = reload_config()
ensure_folio_exists()

## List Tickers

In [ ]:
df = pd.read_excel(config.folio_path, config.tickers_sheet())
print(df)


## Read Folio Transactions

In [ ]:
txns = pd.read_excel(config.folio_path, config.transactions_sheet())
print(txns)

## Nuke Local Data Directory

In [ ]:
import shutil

data_dir = config.project_root / "data"

# Check if the directory exists
if data_dir.exists():
    try:
        # Use shutil.rmtree to delete the directory and its contents
        shutil.rmtree(data_dir)
        print(f"Deleted directory: {data_dir}")
    except PermissionError as e:
        print(f"Permission denied: {e}")
    except FileNotFoundError as e:
        print(f"Directory not found: {e}")
else:
    print(f"Directory does not exist: {data_dir}")